# Procurement Timeline Analysis

This notebook analyzes procurement data and tender information.

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import os

# Create charts directory
os.makedirs("../public/charts", exist_ok=True)

print("✅ Environment ready")

In [ ]:
# Create sample procurement data based on the examples provided
# In a real implementation, this would come from actual data sources

procurement_data = [
    {"id": "N°11", "item": "Equipo de Nefrología", "date": "2023-11-13", "amount": 1500000},
    {"id": "N°10", "item": "Combi Mini Bus", "date": "2023-11-13", "amount": 850000},
    {"id": "N°9", "item": "Camioneta Utilitaria", "date": "2023-11-17", "amount": 1200000},
    {"id": "N°8", "item": "Compactador", "date": "2023-11-17", "amount": 950000},
    {"id": "N°7", "item": "Sistema de Producción y Almacenamiento", "date": "2023-11-27", "amount": 2100000},
    {"id": "N°6", "item": "Equipos de Limpieza Vial", "date": "2023-10-15", "amount": 750000},
    {"id": "N°5", "item": "Mobiliario Urbano", "date": "2023-09-22", "amount": 420000},
    {"id": "N°4", "item": "Material Médico Hospitalario", "date": "2023-08-30", "amount": 310000},
    {"id": "N°3", "item": "Equipos de Cocina Escolar", "date": "2023-07-18", "amount": 180000},
    {"id": "N°2", "item": "Herramientas Agrícolas", "date": "2023-06-05", "amount": 95000},
    {"id": "N°1", "item": "Insumos de Limpieza", "date": "2023-05-12", "amount": 65000}
]

df_proc = pd.DataFrame(procurement_data)
df_proc["date"] = pd.to_datetime(df_proc["date"])

print(f"✅ Created sample procurement data with {len(df_proc)} items")
print("\n📋 Sample data:")
print(df_proc.head())

In [ ]:
# Create timeline visualization
fig = px.scatter(
    df_proc,
    x="date",
    y="item",
    size="amount",
    color="amount",
    hover_data=["id", "amount"],
    title="Public Procurements 2023",
    labels={
        "date": "Date",
        "item": "Item",
        "amount": "Amount (ARS)",
        "id": "Tender ID"
    },
    color_continuous_scale="viridis"
)

fig.update_layout(
    xaxis_title="Date",
    yaxis_title="Procurement Item",
    height=600
)

fig.show()
fig.write_html("../public/charts/procurement_timeline.html")
print("✅ Generated procurement timeline chart")

In [ ]:
# Create bar chart of procurement amounts
fig2 = px.bar(
    df_proc.sort_values("amount", ascending=False),
    x="item",
    y="amount",
    title="Procurement Amounts by Item",
    labels={
        "amount": "Amount (ARS)",
        "item": "Procurement Item"
    },
    color="amount",
    color_continuous_scale="blues"
)

fig2.update_layout(
    xaxis_title="Procurement Item",
    yaxis_title="Amount (ARS)",
    height=500,
    xaxis_tickangle=-45
)

fig2.show()
fig2.write_html("../public/charts/procurement_amounts.html")
print("✅ Generated procurement amounts chart")

# Summary statistics
total_amount = df_proc["amount"].sum()
avg_amount = df_proc["amount"].mean()
max_amount = df_proc["amount"].max()
min_amount = df_proc["amount"].min()

print(f"\n📊 Procurement Summary:")
print(f"   • Total amount: ${total_amount:,.2f} ARS")
print(f"   • Average amount: ${avg_amount:,.2f} ARS")
print(f"   • Highest amount: ${max_amount:,.2f} ARS")
print(f"   • Lowest amount: ${min_amount:,.2f} ARS")
print(f"   • Number of procurements: {len(df_proc)}")

In [ ]:
# Monthly procurement analysis
df_proc["month"] = df_proc["date"].dt.month
df_proc["month_name"] = df_proc["date"].dt.strftime("%B")

# Aggregate by month
monthly_agg = df_proc.groupby(["month", "month_name"]).agg(
    {"amount": ["sum", "count", "mean"]}
).reset_index()

# Flatten column names
monthly_agg.columns = ["month", "month_name", "total_amount", "procurement_count", "avg_amount"]
monthly_agg = monthly_agg.sort_values("month")

# Create monthly trend chart
fig3 = go.Figure()

fig3.add_trace(go.Bar(
    x=monthly_agg["month_name"],
    y=monthly_agg["total_amount"],
    name="Total Amount",
    marker_color="lightblue"
))

fig3.add_trace(go.Scatter(
    x=monthly_agg["month_name"],
    y=monthly_agg["procurement_count"],
    name="Number of Procurements",
    yaxis="y2",
    mode="lines+markers",
    line=dict(color="red", width=3)
))

fig3.update_layout(
    title="Monthly Procurement Activity",
    xaxis_title="Month",
    yaxis=dict(
        title="Total Amount (ARS)",
        tickformat="$,.0f"
    ),
    yaxis2=dict(
        title="Number of Procurements",
        overlaying="y",
        side="right"
    ),
    height=500
)

fig3.show()
fig3.write_html("../public/charts/monthly_procurement_activity.html")
print("✅ Generated monthly procurement activity chart")